In [ ]:
import os
import camb

In [ ]:
pars = camb.CAMBparams()
pars = camb.set_params(H0=67.32, ombh2=0.022383, omch2=0.12011,tau=0.0543,mnu=0.06,omk=0,standard_neutrino_neff=True,  
                       As=2.1005829616811546e-9, ns=0.965, halofit_version='mead', lmax=3500)

In [ ]:
#This is essentially from Oliver's PolySpec Github
transfer_file = 'camb_output3500_Ecorrected.npz'#camb_transfer_3500.npz

if not os.path.exists(transfer_file):  # removed the complication of an overarching root, and it just dumps things out in your directory
    print("Computing transfer functions...")
    
    # Transfer function settings
    pars.DoLensing = True
    pars.BessIntBoost = 30
    pars.KmaxBoost = 3
    pars.IntTolBoost = 4
    pars.TimeStepBoost = 5
    pars.SourcekAccuracyBoost = 5
    pars.BesselBoost = 5
    pars.IntkAccuracyBoost = 5
    pars.AccurateBB = True
    pars.AccurateReionization = True
    pars.AccuratePolarization = True
    pars.set_accuracy(DoLateRadTruncation=False, AccuracyBoost=5, 
                     lSampleBoost=50, lAccuracyBoost=3)

    # Compute transfer functions and spectra    
    results = camb.get_results(pars)
    trans = results.get_cmb_transfer_data()
    ls, qs, DeltasT = trans.get_transfer(source=0)
    ls, qs, DeltasE = trans.get_transfer(source=1)

    # Rescale E mode amplitude
    ls = ls.astype(int)
    prefactorE = np.sqrt((ls + 2) * (ls + 1) * ls * (ls - 1))
    DeltasE *= prefactorE[:,None]

    # Save transfer functions to current directory
    np.savez(transfer_file, l=ls, k=qs, transfer=np.asarray([DeltasT, DeltasE]))
    print(f"Transfer functions saved to {os.path.abspath(transfer_file)}")
else:
    print(f"Using existing transfer file: {os.path.abspath(transfer_file)}")

An Earlier Iteration Where I was saving things as a text file